# Customer Churn Analysis
Churn rate, when applied to a customer base, refers to the proportion of contractual customers or subscribers who leave a supplier during a given time period. It is a possible indicator of customer dissatisfaction, cheaper and/or better offers from the competition, more successful sales and/or marketing by the competition, or reasons having to do with the customer life cycle.

Churn is closely related to the concept of average customer life time. For example, an annual churn rate of 25 percent implies an average customer life of four years. An annual churn rate of 33 percent implies an average customer life of three years. The churn rate can be minimized by creating barriers which discourage customers to change suppliers (contractual binding periods, use of proprietary technology, value-added services, unique business models, etc.), or through retention activities such as loyalty programs. It is possible to overstate the churn rate, as when a consumer drops the service but then restarts it within the same year. Thus, a clear distinction needs to be made between "gross churn", the total number of absolute disconnections, and "net churn", the overall loss of subscribers or members. The difference between the two measures is the number of new subscribers or members that have joined during the same period. Suppliers may find that if they offer a loss-leader "introductory special", it can lead to a higher churn rate and subscriber abuse, as some subscribers will sign on, let the service lapse, then sign on again to take continuous advantage of current specials. https://en.wikipedia.org/wiki/Churn_rate

In [1]:
%%capture

# Get our favorite packages from PyPI
! pip install cufflinks

# Import pre-installed packages
import numpy as np
import pandas as pd

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

In [2]:
# Load our favorite visualization library
import os
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Sign into Plotly with masked, encrypted API key

myPlotlyKey = os.environ['SECRET_ENV_BRETTS_PLOTLY_KEY']
py.sign_in(username='bretto777',api_key=myPlotlyKey)


In [3]:
# Load some data
churnDF = pd.read_csv('https://s3-us-west-1.amazonaws.com/dsclouddata/home/jupyter/churn_train.csv', delimiter=',')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=False, value='Retain')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=True, value='Churn')
churnDFs = churnDF.sample(frac=0.07) # Sample for speedy viz
churnDF.head(10)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,ND,84,415,400-7253,no,yes,33,159.1,106,27.05,...,101,12.73,213.4,108,9.60,13.0,18,3.51,1,Retain
1,RI,117,408,370-5042,no,yes,13,207.6,65,35.29,...,77,12.98,232.8,95,10.48,9.7,3,2.62,1,Retain
2,VA,132,510,343-4696,no,no,0,81.1,86,13.79,...,72,20.84,237.0,115,10.67,10.3,2,2.78,0,Retain
3,OK,121,408,364-2495,no,yes,31,237.1,63,40.31,...,117,17.48,196.7,85,8.85,10.1,5,2.73,4,Retain
4,ME,205,510,413-4039,no,yes,24,175.8,139,29.89,...,98,13.18,180.7,64,8.13,7.8,5,2.11,2,Retain
5,MT,116,415,384-5907,no,yes,35,182.8,122,31.08,...,119,18.08,193.8,103,8.72,11.0,2,2.97,1,Retain
6,MN,113,408,417-5146,no,no,0,158.9,137,27.01,...,109,20.64,247.8,97,11.15,6.5,4,1.76,0,Retain
7,MO,74,415,421-2955,no,no,0,172.1,105,29.26,...,99,17.99,182.2,105,8.20,11.6,6,3.13,1,Retain
8,MI,112,510,420-1383,no,no,0,243.4,77,41.38,...,97,15.48,259.2,94,11.66,12.8,2,3.46,1,Retain
9,UT,68,415,403-8916,no,no,0,213.9,112,36.36,...,100,22.14,233.8,97,10.52,8.4,3,2.27,1,Churn


In [4]:
# separate the calls data for plotting


churnDFs = churnDFs[['Account Length','Day Calls','Eve Calls','CustServ Calls','Churn']]

# Create scatter plot matrix of call data
splom = ff.create_scatterplotmatrix(churnDFs, diag='histogram', index='Churn',  
                                  colormap= dict(
                                      Churn = '#9CBEF1',
                                      Retain = '#04367F'
                                      ),
                                  colormap_type='cat',
                                  height=560, width=650,
                                  size=4, marker=dict(symbol='circle'))
py.iplot(splom)

In [8]:

import h2o
from h2o.automl import H2OAutoML
from __future__ import print_function
h2o.connect(ip="35.196.227.23")

Connecting to H2O server at http://35.196.227.23:54321... successful.


H2O cluster uptime:,7 mins 35 secs
H2O cluster version:,3.13.0.318
H2O cluster version age:,4 months and 12 days !!!
H2O cluster name:,olmsteadbrett
H2O cluster total nodes:,1
H2O cluster free memory:,1.606 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"
H2O connection url:,http://35.196.227.23:54321
H2O connection proxy:,None


<H2OConnection to http://35.196.227.23:54321, session _sid_b5bf>

In [6]:
# Split data into training and testing frames

from sklearn import cross_validation
from sklearn.model_selection import train_test_split

training, testing = train_test_split(churnDF, train_size=0.8, stratify=churnDF["Churn"], random_state=9)
train = h2o.H2OFrame(python_obj=training).drop("State")
test = h2o.H2OFrame(python_obj=testing).drop("State")

# Set predictor and response variables
y = "Churn"
x = train.columns
x.remove(y)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Automatic Machine Learning
enter some text here

In [33]:
# Run AutoML for 20 minutes or until leader fails to improve after 5 rounds
autoModel = H2OAutoML(max_models = 11, stopping_rounds=5, stopping_tolerance=0.001)
autoModel.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


## Leaderboard

In [34]:
leaders = autoModel.leaderboard.
leaders

model_id,auc,logloss
GBM_grid_0_AutoML_20171107_185141_model_3,0.940261,0.164768
GBM_grid_0_AutoML_20171107_182007_model_0,0.936854,0.171059
GBM_grid_0_AutoML_20171107_185141_model_5,0.934252,0.151757
GBM_grid_0_AutoML_20171107_185141_model_2,0.931118,0.19973
GBM_grid_0_AutoML_20171107_185141_model_0,0.930259,0.357099
GBM_grid_0_AutoML_20171107_185141_model_1,0.92846,0.203286
GBM_grid_0_AutoML_20171107_185141_model_6,0.927016,0.202103
XRT_0_AutoML_20171107_185141,0.926648,0.315412
GBM_grid_0_AutoML_20171107_182007_model_2,0.92132,0.177214
DRF_0_AutoML_20171107_185141,0.919481,0.36809


# Variable Importances
Below we plot variable importances as reported by the best performing algo in the ensemble.

In [35]:
importances = h2o.get_model(leaders[0, 0]).varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

# Super Model vs the Base models
This plot shows the ROC curves for the Super Model, the Best Base Model, and 9 next best models in the ensemble. 

In [37]:
Model0 = np.array(h2o.get_model(leaders[0,0]).roc(xval=True))
Model1 = np.array(h2o.get_model(leaders[1,0]).roc(xval=True))
Model2 = np.array(h2o.get_model(leaders[2,0]).roc(xval=True))
Model3 = np.array(h2o.get_model(leaders[3,0]).roc(xval=True))
Model4 = np.array(h2o.get_model(leaders[4,0]).roc(xval=True))
Model5 = np.array(h2o.get_model(leaders[5,0]).roc(xval=True))
Model6 = np.array(h2o.get_model(leaders[6,0]).roc(xval=True))
Model7 = np.array(h2o.get_model(leaders[7,0]).roc(xval=True))
Model8 = np.array(h2o.get_model(leaders[8,0]).roc(xval=True))
Model9 = np.array(h2o.get_model(leaders[9,0]).roc(xval=True))

layout = go.Layout(autosize=False, width=725, height=575,  xaxis=dict(title='False Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')), 
                                                           yaxis=dict(title='True Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')))

traceChanceLine = go.Scatter(x = [0,1], y = [0,1], mode = 'lines+markers', name = 'chance', line = dict(color = ('rgb(136, 140, 150)'), width = 4, dash = 'dash'))

Model0Trace = go.Scatter(x = Model0[0], y = Model0[1], mode = 'lines', name = 'Model 0', line = dict(color = ('rgb(26, 58, 126)'), width = 3))
Model1Trace = go.Scatter(x = Model1[0], y = Model1[1], mode = 'lines', name = 'Model 1', line = dict(color = ('rgb(135, 160, 216)'), width = 3))
Model2Trace = go.Scatter(x = Model2[0], y = Model2[1], mode = 'lines', name = 'Model 2', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model3Trace = go.Scatter(x = Model3[0], y = Model3[1], mode = 'lines', name = 'Model 3', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model4Trace = go.Scatter(x = Model4[0], y = Model4[1], mode = 'lines', name = 'Model 4', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model5Trace = go.Scatter(x = Model5[0], y = Model5[1], mode = 'lines', name = 'Model 5', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model6Trace = go.Scatter(x = Model6[0], y = Model6[1], mode = 'lines', name = 'Model 6', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model7Trace = go.Scatter(x = Model7[0], y = Model7[1], mode = 'lines', name = 'Model 7', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model8Trace = go.Scatter(x = Model8[0], y = Model8[1], mode = 'lines', name = 'Model 8', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model9Trace = go.Scatter(x = Model9[0], y = Model9[1], mode = 'lines', name = 'Model 9', line = dict(color = ('rgb(156, 190, 241)'), width = 1))

fig = go.Figure(data=[Model0Trace,Model1Trace,Model2Trace,Model3Trace,Model4Trace,Model5Trace,Model6Trace,Model7Trace,Model8Trace,Model9Trace,traceChanceLine], layout=layout)


py.iplot(fig)


# Confusion Matrix

In [40]:
cm = h2o.get_model(leaders[0, 0]).confusion_matrix()
cm = cm.table.as_data_frame()
cm
confusionMatrix = ff.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)


# Business Impact Matrix

Weighting Predictions With a Dollar Value
-   Correctly predicting retain: `+$5`
-   Correctly predicting churn: `+$75`
-   Incorrectly predicting retain: `-$150`
-   Incorrectly predicting churn: `-$1.5`

    

In [41]:
CorrectPredictChurn = cm.loc[0,'Churn']
CorrectPredictChurnImpact = 75
cm1 = CorrectPredictChurn*CorrectPredictChurnImpact

IncorrectPredictChurn = cm.loc[1,'Churn']
IncorrectPredictChurnImpact = -5
cm2 = IncorrectPredictChurn*IncorrectPredictChurnImpact

IncorrectPredictRetain = cm.loc[0,'Retain']
IncorrectPredictRetainImpact = -150
cm3 = IncorrectPredictRetain*IncorrectPredictRetainImpact

CorrectPredictRetain = cm.loc[0,'Retain']
CorrectPredictRetainImpact = 5
cm4 = IncorrectPredictRetain*CorrectPredictRetainImpact


data_matrix = [['Business Impact', '($) Predicted Churn', '($) Predicted Retain', '($) Total'],
               ['($) Actual Churn', cm1, cm3, '' ],
               ['($) Actual Retain', cm2, cm4, ''],
               ['($) Total', cm1+cm2, cm3+cm4, cm1+cm2+cm3+cm4]]

impactMatrix = ff.create_table(data_matrix, height_constant=20, hoverinfo='weight')
impactMatrix.layout.height=300
impactMatrix.layout.width=800
impactMatrix.layout.font.size=17
py.iplot(impactMatrix)

In [42]:
print("Total customers evaluated: 534")

Total customers evaluated: 534


In [43]:
print("Total value created by the model: $" + str(cm1+cm2+cm3+cm4))

Total value created by the model: $9465.0


In [44]:
print("Total value per customer: $" +str(round(((cm1+cm2+cm3+cm4)/534),3)))

Total value per customer: $17.725


In [47]:
os.getcwd()

'/home/datascience/Predict-Churn'

In [52]:
# Save the Model



path = h2o.save_model(model=h2o.get_model(leaders[0, 0]), force=True, path="/home/datascience/Predict-Churn/")
#LoadedEnsemble = h2o.load_model(path='H2O-AutoML-Model')

H2OServerError: HTTP 500 Server Error:
Server error water.api.FSIOException:
  Error: FS IO Failure: 
 accessed path : file:/home/datascience/Predict-Churn/GBM_grid_0_AutoML_20171107_185141_model_3 caused by: /home/datascience/Predict-Churn/GBM_grid_0_AutoML_20171107_185141_model_3 (No such file or directory)
  Request: None
